In [59]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))
import numpy as np
import nashpy as nash
from IPython.display import display
import support_enumeration as se
import BNE as BNE
import IESDS as IESDS

### 1.1

### 1.2

In [60]:
for theta in [-1/10, 0, 1/10]:
    U1 = np.array([[(1/2) + theta, 1], 
                   [1, (1/2) - theta]])
    U2 = np.array([[-(1/2) - theta, -1], 
                   [-1, -(1/2) + theta]])

    NE = se.msne(U1, U2)

    print(NE)

[(array([0.4, 0.6]), array([0.4, 0.6]))]
[(array([0.5, 0.5]), array([0.5, 0.5]))]
[(array([0.6, 0.4]), array([0.6, 0.4]))]


### 1.3

Svaret i paint.

### 1.4

In [61]:
theta1 = -1/10
theta2 = 1/10

U1_A = np.array([[(1/2) + theta1, 1], 
                   [1, (1/2) - theta1]])
U1_B = np.array([[-(1/2) + theta2, 1], 
                   [1, (1/2) - theta2]])


U2_A = np.array([[-(1/2) - theta1, -1], 
                   [-1, -(1/2) + theta1]])
U2_B = np.array([[-(1/2) - theta2, -1], 
                   [-1, -(1/2) + theta2]])


In [62]:
U1 = [U1_A ,U1_B]
U2 = [U2_A,U2_B]

action = ['L','R']

BU1, BU2 = BNE.compute_full_matrix(U1,U2,(1/2), action, action)

#IESDS
# U1, U2, aa1, aa2 = IESDS.IESDS(BU1[0], BU2[0])

result = BNE.compute_bne(BU1[0],BU2[0])

print(f"p = {1/2}: BNE {result}")
display(BU1[1])
display(BU2[1])

p = 0.5: BNE []


c:\Users\elias\anaconda3\Lib\site-packages\nashpy\algorithms\support_enumeration.py:260: RuntimeWarning: 
An even number of (0) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


,LL,LR,RL,RR
L,0.0,0.7,0.3,1.0
R,1.0,0.7,0.8,0.5


,LL,LR,RL,RR
L,-0.5,-0.7,-0.8,-1.0
R,-1.0,-0.7,-0.8,-0.5


In [63]:
U1 = [U1_A ,U1_B]
U2 = [U2_A,U2_B]

action = ['L','R']

BU1, BU2 = BNE.compute_full_matrix(U1,U2,(1/2), action, action)

#IESDS
U1, U2, aa1, aa2 = IESDS.IESDS(BU1[0], BU2[0])

result = BNE.compute_bne(U1,U2)

print(f"p = {1/2}: BNE {result}")
display(BU1[1])
display(BU2[1])

a=2 is strictly dominated by a=1
No further strategies to delete after 1 iterations
p = 0.5: BNE []


c:\Users\elias\anaconda3\Lib\site-packages\nashpy\algorithms\support_enumeration.py:260: RuntimeWarning: 
An even number of (0) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


,LL,LR,RL,RR
L,0.0,0.7,0.3,1.0
R,1.0,0.7,0.8,0.5


,LL,LR,RL,RR
L,-0.5,-0.7,-0.8,-1.0
R,-1.0,-0.7,-0.8,-0.5


In [67]:
import numpy as np

def find_bne_intervals(U1_types, U2_types, tol=1e-8):
    """
    U1_types, U2_types: lister af 2×2 NumPy-arrays, én for hver af de 2 typer.
    Returnerer liste af (mapping, p_low, p_high), hvor mapping=(a_-0.1, a_+0.1)
    med 0=L,1=R, og [p_low,p_high] er det interval af p med BNE.
    """
    n_types = len(U1_types)      # =2
    nA1, nA2 = U1_types[0].shape # = (2,2)
    prior = [1/n_types]*n_types  # = [0.5,0.5]
    results = []

    # 1) Gå igennem alle rene planer for Sif: (0,0),(0,1),(1,0),(1,1)
    for plan in [(i,j) for i in (0,1) for j in (0,1)]:
        # 2) Beregn Ava's interim-payoff for ren L (a1=0) og R (a1=1)
        EU = []
        for a1 in (0,1):
            val = sum(prior[t] * U1_types[t][a1, plan[t]]
                      for t in (0,1))
            EU.append(val)
        # 3) Ava skal være indifferent for at blande:
        if abs(EU[0] - EU[1]) > tol:
            continue

        # 4) Best-response for Sif giver lineære uligheder i p:
        p_low, p_high = 0.0, 1.0
        for t in (0,1):
            a_star = plan[t]
            for a_alt in (0,1):
                if a_alt == a_star: continue
                # EU_S(a_star) >= EU_S(a_alt):
                # p*(U2[t][0,a*]−U2[t][0,a_alt]) + (1−p)*(U2[t][1,a*]−U2[t][1,a_alt]) ≥ 0
                α = U2_types[t][0,a_star] - U2_types[t][0,a_alt]
                β = U2_types[t][1,a_star] - U2_types[t][1,a_alt]
                diff = α - β
                if abs(diff) < tol:
                    if β < -tol:
                        p_low, p_high = 1.0, 0.0
                else:
                    thresh = -β/diff
                    if diff > 0:
                        p_low = max(p_low, thresh)
                    else:
                        p_high = min(p_high, thresh)
            if p_low > p_high:
                break

        if p_low <= p_high:
            results.append((plan, max(0,p_low), min(1,p_high)))

    return results

# --- Eksempel på jeres spil ---
theta1, theta2 = -0.1, +0.1
U1_A = np.array([[(1/2) + theta1, 1], 
                   [1, (1/2) - theta1]])
U1_B = np.array([[-(1/2) + theta2, 1], 
                   [1, (1/2) - theta2]])


U2_A = np.array([[-(1/2) - theta1, -1], 
                   [-1, -(1/2) + theta1]])
U2_B = np.array([[-(1/2) - theta2, -1], 
                   [-1, -(1/2) + theta2]])

bne = find_bne_intervals([U1_A, U1_B], [U2_A, U2_B])
for plan, lo, hi in bne:
    plan_str = tuple('L' if x==0 else 'R' for x in plan)
    print(f"Sif spiller {plan_str} ⇒ BNE for p ∈ [{lo:.3f}, {hi:.3f}]")

# Forventet output:
# Sif spiller ('L','R') ⇒ BNE for p ∈ [0.400, 0.600]


Sif spiller ('L', 'R') ⇒ BNE for p ∈ [0.400, 0.600]
